In [ ]:
import geopandas as gpd
from shapely.geometry.multilinestring import MultiLineString

lines = gpd.read_file('water_lines.gpkg')

polygons = gpd.read_file('water_polygons.gpkg')



Из базовых идей:
1. Продлеваем реки немного и sjoin на пересечение с водой, фильтруем воду, геомку возвращаем.
2. Для больших озёр, не рек строим как у стива разбиение полигона - получаем точки и ещё раз вороного сверху, получаем озеро, теоретически разбитое ровненько по блокам одинаковым, в теории
3.

In [ ]:
from shapely import LineString
import math
import tifffile as tiff
import numpy as np
from skimage import measure
from skimage.filters import gaussian

in_path = "N59E030_FABDEM_V1-2.tif"
step_m = 3


def extend_linestring(line: LineString, distance: float = 1.0) -> LineString:
    if len(line.coords) < 2:
        raise ValueError("LineString must have at least two points.")

    x0, y0 = line.coords[0]
    x1, y1 = line.coords[1]
    dx0 = x0 - x1
    dy0 = y0 - y1
    L0 = math.hypot(dx0, dy0)
    new_start = (x0 + dx0 / L0 * distance, y0 + dy0 / L0 * distance)

    xN1, yN1 = line.coords[-2]
    xN, yN = line.coords[-1]
    dx1, dy1 = xN - xN1, yN - yN1
    L1 = math.hypot(dx1, dy1) or 1.0
    new_end = (xN + dx1 / L1 * distance, yN + dy1 / L1 * distance)

    return LineString([new_start, *list(line.coords[1:-1]), new_end])


with tiff.TiffFile(in_path) as tif:
    page = tif.pages[0]
    tags = page.tags
    scale = tags.get("ModelPixelScaleTag")
    tie = tags.get("ModelTiepointTag")
    if scale is None or tie is None:
        raise ValueError("Не найдены GeoTIFF-теги ModelPixelScaleTag / ModelTiepointTag")

    sx, sy = float(scale.value[0]), float(scale.value[1])
    i0, j0, _, X0, Y0, _ = tie.value[:6]
    i0, j0, X0, Y0 = float(i0), float(j0), float(X0), float(Y0)

    width = page.imagewidth
    height = page.imagelength

    hx = sx * 0.5
    hy = sy * 0.5

    x_min = X0 - hx
    x_max = X0 + (width - 0.5) * sx
    y_max = Y0 + hy
    y_min = Y0 - (height - 0.5) * sy

    arr = page.asarray()
    if arr.ndim == 3:
        arr = arr[0]

    nd_tag = page.tags.get("GDAL_NODATA")
    nodata = float(nd_tag.value) if nd_tag is not None else -9999.0

arr = np.where(arr == nodata, np.nan, arr).astype(float)

zmin = np.nanmin(arr)
zmax = np.nanmax(arr)

z_start = math.floor(zmin / step_m) * step_m
z_stop = math.ceil(zmax / step_m) * step_m
levels = np.arange(z_start, z_stop + step_m, step_m)

arr = np.round(arr / step_m) * step_m

lines = []
lvls = []
for lev in levels:
    contours = measure.find_contours(arr, level=lev)
    for cnt in contours:
        rows = cnt[:, 0]
        cols = cnt[:, 1]
        X = x_min + (cols - i0) * sx
        Y = y_max - (rows - j0) * sy

        coords = list(zip(X, Y))
        if len(coords) < 2:
            continue
        line = LineString(coords)
        if line.is_empty:
            continue
        if not line.is_closed:
            line = extend_linestring(line, 0.001)

        lines.append(line)
        lvls.append(lev)

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]

gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326).to_file(
    'test_poly_high.gpkg')

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]

polygons_gdf = gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326)
# lines_gdf = gpd.GeoDataFrame(lvls, geometry=lines, crs=4326, columns=['level'])


In [ ]:
rep_pts = polygons_gdf.representative_point()
col = np.round((rep_pts.x.values - x_min) / sx + i0).astype(int)
row = np.round((y_max - rep_pts.y.values) / sy + j0).astype(int)

h, w = arr.shape
col = np.clip(col, 0, w - 1)
row = np.clip(row, 0, h - 1)

vals = arr[row, col]

polygons_gdf["level"] = vals

In [ ]:
polygons_gdf.to_file('test_poly_high.gpkg')

In [ ]:
gpd.GeoDataFrame(lvls, geometry=lines, crs=4326, columns=['level']).to_file('test_lines_1m.gpkg')

# Градиент


In [ ]:
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
from pyproj import CRS, Transformer
from shapely import LineString
import math
import tifffile as tiff
import numpy as np
from skimage import measure
from skimage.filters import gaussian

in_path = "N59E030_FABDEM_V1-2.tif"
step_m = 1


def extend_linestring(line: LineString, distance: float = 1.0) -> LineString:
    if len(line.coords) < 2:
        raise ValueError("LineString must have at least two points.")

    x0, y0 = line.coords[0]
    x1, y1 = line.coords[1]
    dx0 = x0 - x1
    dy0 = y0 - y1
    L0 = math.hypot(dx0, dy0)
    new_start = (x0 + dx0 / L0 * distance, y0 + dy0 / L0 * distance)

    xN1, yN1 = line.coords[-2]
    xN, yN = line.coords[-1]
    dx1, dy1 = xN - xN1, yN - yN1
    L1 = math.hypot(dx1, dy1) or 1.0
    new_end = (xN + dx1 / L1 * distance, yN + dy1 / L1 * distance)

    return LineString([new_start, *list(line.coords[1:-1]), new_end])


with tiff.TiffFile(in_path) as tif:
    page = tif.pages[0]
    tags = page.tags
    scale = tags.get("ModelPixelScaleTag")
    tie = tags.get("ModelTiepointTag")
    if scale is None or tie is None:
        raise ValueError("Не найдены GeoTIFF-теги ModelPixelScaleTag / ModelTiepointTag")

    sx, sy = float(scale.value[0]), float(scale.value[1])
    i0, j0, _, X0, Y0, _ = tie.value[:6]
    i0, j0, X0, Y0 = float(i0), float(j0), float(X0), float(Y0)

    width = page.imagewidth
    height = page.imagelength

    hx = sx * 0.5
    hy = sy * 0.5

    x_min = X0 - hx
    x_max = X0 + (width - 0.5) * sx
    y_max = Y0 + hy
    y_min = Y0 - (height - 0.5) * sy

    arr = page.asarray()
    if arr.ndim == 3:
        arr = arr[0]

    nd_tag = page.tags.get("GDAL_NODATA")
    nodata = float(nd_tag.value) if nd_tag is not None else -9999.0

arr = np.where(arr == nodata, np.nan, arr).astype(float)

utm_infos = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=float(x_min),
        south_lat_degree=float(y_min),
        east_lon_degree=float(x_max),
        north_lat_degree=float(y_max),
    ),
)
if not utm_infos:
    raise RuntimeError("Не удалось подобрать UTM CRS для данного экстента.")
utm_crs = CRS.from_epsg(utm_infos[0].code)
to_utm = Transformer.from_crs(4326, utm_crs, always_xy=True)  # lon/lat -> UTM

lon_c = (x_min + x_max) / 2.0
lat_c = (y_min + y_max) / 2.0
x1, y1 = to_utm.transform(lon_c, lat_c)
x2, y2 = to_utm.transform(lon_c + sx, lat_c)
x3, y3 = to_utm.transform(lon_c, lat_c - sy)
dx_m = abs(x2 - x1)  # метров/пиксель по X
dy_m = abs(y3 - y1)

arr_gauss = gaussian(arr, sigma=1.0, preserve_range=True)

dzdy, dzdx = np.gradient(arr_gauss, dy_m, dx_m)
grad = np.hypot(dzdx, dzdy)
slope_deg = np.degrees(np.arctan(grad))

zmin = np.nanmin(slope_deg)
zmax = np.nanmax(slope_deg)

z_start = math.floor(zmin / step_m) * step_m
z_stop = math.ceil(zmax / step_m) * step_m
levels = np.arange(z_start, z_stop + step_m, step_m)
slope_deg = np.ceil(slope_deg / step_m) * step_m

slope_lines = []
slope_levels = []

slope_fill = np.nanmin(slope_deg) - 9999.0
slope_for_contours = np.where(np.isnan(slope_deg), slope_fill, slope_deg)
for lev in levels:
    contours = measure.find_contours(slope_for_contours, level=lev)
    for cnt in contours:
        rows = cnt[:, 0]
        cols = cnt[:, 1]
        lon = x_min + (cols - i0) * sx
        lat = y_max - (rows - j0) * sy
        X, Y = lon, lat
        if len(X) < 2:
            continue
        line = LineString(zip(X, Y))
        if line.is_empty:
            continue
        if not line.is_closed:
            line = extend_linestring(line, 0.001)
        slope_lines.append(line)
        slope_levels.append(float(lev))


dzdy, dzdx = np.gradient(arr_gauss, dy_m, dx_m)
aspect_rad = np.arctan2(-dzdx, dzdy)
aspect_deg = (np.degrees(aspect_rad) + 360.0) % 360.0

degree_step = 45

aspect_bins_deg  = np.arange(0,360+degree_step,degree_step)
aspect_class = np.digitize(aspect_deg, aspect_bins_deg, right=False) - 1
aspect_classes = np.digitize(aspect_bins_deg, aspect_bins_deg, right=False) - 1

aspect_lines = []
aspect_degrees = []


for n_class in aspect_classes:
    contours = measure.find_contours(aspect_class, level=n_class)
    print(n_class, len(contours))
    for cnt in contours:
        rows = cnt[:, 0]
        cols = cnt[:, 1]
        lon = x_min + (cols - i0) * sx
        lat = y_max - (rows - j0) * sy
        X, Y = lon, lat
        if len(X) < 2:
            continue
        line = LineString(zip(X, Y))
        if line.is_empty:
            continue
        if not line.is_closed:
            line = extend_linestring(line, 0.001)
        aspect_lines.append(line)
        aspect_degrees.append(n_class)


In [ ]:
np.digitize(aspect_bins_deg, aspect_bins_deg, right=False) - 1

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = slope_lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]

polygons_gdf = gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326)

rep_pts = polygons_gdf.representative_point()
col = np.round((rep_pts.x.values - x_min) / sx + i0).astype(int)
row = np.round((y_max - rep_pts.y.values) / sy + j0).astype(int)

h, w = arr.shape
col = np.clip(col, 0, w - 1)
row = np.clip(row, 0, h - 1)

vals = slope_for_contours[row, col]

polygons_gdf["level"] = vals


In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

lines_w_bounds = aspect_lines + [LineString(Polygon.from_bounds(x_min, y_min, x_max, y_max).exterior)]

polygons_gdf = gpd.GeoDataFrame(geometry=list(polygonize(node(MultiLineString(lines_w_bounds)))), crs=4326)

rep_pts = polygons_gdf.representative_point()
col = np.round((rep_pts.x.values - x_min) / sx + i0).astype(int)
row = np.round((y_max - rep_pts.y.values) / sy + j0).astype(int)

h, w = arr.shape
col = np.clip(col, 0, w - 1)
row = np.clip(row, 0, h - 1)

vals = aspect_deg[row, col]

polygons_gdf["level"] = vals


In [ ]:

polygons_gdf.to_file('test_aspect.gpkg')

In [ ]:
polygons_gdf.explore(column="level")

In [ ]:
from shapely.ops import polygonize, unary_union
from shapely import build_area, node, MultiLineString, Polygon
import geopandas as gpd

gpd.GeoDataFrame(slope_levels, geometry=slope_lines, crs=4326, columns=['level']).explore(column='level')

# Экспозиция(направление наклона)
